In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}


<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams

In [3]:
%matplotlib inline

In [4]:
TEXT_COLOUR = {
    'PURPLE':'\033[95m',
    'CYAN':'\033[96m',
    'DARKCYAN':'\033[36m',
    'BLUE':'\033[94m',
    'GREEN':'\033[92m',
    'YELLOW':'\033[93m',
    'RED':'\033[91m',
    'BOLD':'\033[1m',
    'UNDERLINE':'\033[4m',
    'END':'\033[0m'
}

def print_bold(*msgs):
    print(TEXT_COLOUR['BOLD'])
    print(*msgs)
    print(TEXT_COLOUR['END'])

def print_green(*msgs):
    print(TEXT_COLOUR['GREEN'])
    print(*msgs)
    print(TEXT_COLOUR['END'])

def print_error(*msgs):
    print(TEXT_COLOUR['RED'])
    print(*msgs)
    print(TEXT_COLOUR['END'])

def wrap_green(msg):
    return TEXT_COLOUR['GREEN'] + msg + TEXT_COLOUR['END']

def wrap_red(msg):
    return TEXT_COLOUR['RED'] + msg + TEXT_COLOUR['END']

def up_down_str(val):
    msg = str(val)
    if val > 0:
        msg = wrap_green(msg)
    elif val < 0:
        msg = wrap_red(msg)
    return msg

In [5]:
exp='roberta-large'
num_layers = 24

In [6]:
# tasks = ["CoLA","SST-2","MRPC","STS-B","QQP","MNLI", "MNLI-MM", "QNLI","RTE"]
tasks = ["CoLA","SST-2","MRPC","STS-B","RTE"]

metrics = {
    "CoLA":["mcc"],
    "MNLI":["acc"],
    "MNLI-MM":["acc"],
    "MRPC":["f1"],
    "QNLI":["acc"],
    "QQP":["f1"],
    "RTE":["acc"],
    "SST-2":["acc"],
    "STS-B":["spearmanr"],
    "WNLI":["acc"] #temp
}

reported_in_paper = {
    "CoLA":0.00,
    "MNLI":0.00,
    "MNLI-MM":0.0,
    "MRPC":0.00,
    "QNLI":0.00,
    "QQP":0.00,
    "RTE":0.00,
    "SST-2":0.00,
    "STS-B":0.00,
    "WNLI":0.00
}

In [7]:

def get_average_val(lines):
    reported = []
    for line in lines:
#         print('\t', line)
        val = float(line.split()[1])
        if val != 0:
            reported.append(val)
    out = 0
    if len(reported) != 0:
        reported.sort(reverse = True)
        candidates = [reported[0]]
        for j in range(1, len(reported)):
            if reported[j] > 0.9 * reported[0]:
                candidates.append(reported[j])
        out = np.mean(candidates)
        
    return out


In [8]:
results = {}

for task in tasks:
    task_results = {}
    task_metrics = metrics[task]
    for metric in task_metrics:
        
        # base metrics
        f=open(f"../../mt_dnn_exp_results/{exp}/{task}/base-{metric}.txt", "r")
        lines = f.read().splitlines()
        task_results[f'base-{metric}'] = get_average_val(lines)
        
        # no layer metrics
        
        fine_tuning_metrics = []
        f=open(f"../../mt_dnn_exp_results/{exp}/{task}/no_layer-{metric}.txt", "r")

        lines = f.read().splitlines()
        fine_tuning_metrics.append(get_average_val(lines))
        
        # fine-tuned metrics
        
        log_file_prefix=''
        for i in reversed(range(int(num_layers/2), num_layers)):
            log_file_prefix += str(i)
            f=open(f"../../mt_dnn_exp_results/{exp}/{task}/{log_file_prefix}-{metric}.txt", "r")
            lines = f.read().splitlines()
            fine_tuning_metrics.append(get_average_val(lines))
            
            log_file_prefix +='_'
        
        task_results[f'{metric}'] = list(reversed(fine_tuning_metrics))
        
    results[task] = task_results

In [9]:
x_axis = []

for i in range(int(num_layers/2), num_layers):
    x_axis.append(str(i))

x_axis.append("none")

In [10]:
def draw_graph(task, y_label, paper, base, reported):
    plt.figure(figsize=(10,6))
    plt.plot(x_axis, reported)
    
    plt.xlabel("layers")
    plt.ylabel(y_label)
    
    if paper == 0.0:    
        gap = max(reported) - min(reported)
        top = max(max(reported), base) + (gap*0.2)
        bottom = min(min(reported), base) - (gap*0.2)
    
        plt.ylim(bottom, top)

        plt.axhline(y=base, linestyle='--', c='green')
    else:
        gap = max(reported) - min(reported)
        top = max(max(reported), base, paper) + (gap*0.2)
        bottom = min(min(reported), base, paper) - (gap*0.2)
    
        plt.ylim(bottom, top)

        plt.axhline(y=base, linestyle='--', c='green')
        plt.axhline(y=paper, linestyle='--', c='red')
    
    plt.title(f'{exp}-{task} ({round(base,4)})')
    plt.savefig(f'images/{exp}/{task}', format='png', bbox_inches='tight')
    plt.show()

In [11]:
for task in tasks:
    task_results = results[task]
    task_metrics = metrics[task]
    for metric in task_metrics:
        reported = task_results[metric]
        base = task_results[f'base-{metric}']
        print_bold(task, metric, ': b -', round(base * 100, 2), 'h -',round(task_results[metric][0] * 100, 2), 'n -', round(task_results[metric][-1] * 100, 2))


CoLA mcc : b - 65.91 h - 60.1 n - 0


SST-2 acc : b - 95.55 h - 95.74 n - 79.27


MRPC f1 : b - 92.79 h - 92.64 n - 81.22


STS-B spearmanr : b - 91.91 h - 90.95 n - 11.19


RTE acc : b - 85.05 h - 83.03 n - 48.74



In [12]:
import copy 

layer_90 = []
layer_95 = []

threshold_90 = 0.9
threshold_95 = 0.95
x_axis.reverse()

for task in tasks:
#     print_bold(task)
    task_results = results[task]
    task_metrics = metrics[task]
    for metric in task_metrics:
        base = task_results[f'base-{metric}']
        reported = copy.deepcopy(task_results[metric])
        reported.reverse()
        
        flag_90 = True
        flag_95 = True

        for ind, val in enumerate(reported):

            if val/base > threshold_90 and flag_90:
                flag_90 = False
                layer_90.append(ind)
                results[task]['90%'] = ind

            if val/base > threshold_95 and flag_95:
                flag_95 = False
                layer_95.append(ind)
                results[task]['95%'] = ind

        if flag_90:
            print(task, "Fails to achieve 90% threshold", reported[-1]/base)
            layer_90.append(len(reported)-1)
            results[task]['90%'] = "-"

        if flag_95:
            print(task, "Fails to achieve 95% threshold", reported[-1]/base)
            layer_95.append(len(reported)-1)
            results[task]['95%'] = "-"


            
print(x_axis)
            
            
print(layer_90)
min_layer_ind_90 = max(layer_90)
print("layer_90 ", min_layer_ind_90, 'layer:', x_axis[min_layer_ind_90], round((1-(min_layer_ind_90/num_layers)) * 100, 2), '%')

print(layer_95)
min_layer_ind_95 = max(layer_95)
print("layer_95 ", min_layer_ind_95, 'layer:', x_axis[min_layer_ind_95], round((1-(min_layer_ind_95/num_layers)) * 100, 2), '%')


firsts = []
seconds = []
    
for task in tasks:
    task_results = results[task]
    task_metrics = metrics[task]
    for metric in task_metrics:
        base = task_results[f'base-{metric}']
        reported = copy.deepcopy(task_results[metric])
        reported.reverse()
        
        if task != "CoLA":
            first = round(100*reported[0]/base, 2)
            second = round(100*reported[1]/base, 2)
            firsts.append(first)
            seconds.append(second)
            
        print_bold(task, base)
        print('\t90', reported[min_layer_ind_90], round(reported[min_layer_ind_90]/base * 100, 2))
        print('\t95', reported[min_layer_ind_95], round(reported[min_layer_ind_95]/base * 100, 2))
        
print_bold(len(firsts), np.mean(firsts), np.mean(seconds), round(np.mean(seconds) - np.mean(firsts),2))

CoLA Fails to achieve 95% threshold 0.9117659837539205
['none', '23', '22', '21', '20', '19', '18', '17', '16', '15', '14', '13', '12']
[5, 1, 1, 2, 7]
layer_90  7 layer: 17 70.83 %
[12, 1, 5, 5, 10]
layer_95  12 layer: 12 50.0 %

CoLA 0.6591311828616843

	90 0.6149273544793329 93.29
	95 0.6009733913647688 91.18

SST-2 0.9554663608562691

	90 0.9510703363914371 99.54
	95 0.9573776758409785 100.2

MRPC 0.9279490864819039

	90 0.9165651138396147 98.77
	95 0.926364166287435 99.83

STS-B 0.9191104104578683

	90 0.8982840764123274 97.73
	95 0.9095195316649449 98.96

RTE 0.8505415162454874

	90 0.7689530685920577 90.41
	95 0.8303249097472926 97.62

4 59.99250000000001 86.72 26.73



In [13]:
for task in ["STS-B"]:
    task_results = results[task]
    task_metrics = metrics[task]
    for metric in task_metrics:
        
        print(task_results[metric][-1])
        print(task_results[metric][-2])

0.11190994192398618
0.8238397398130405


In [14]:
latex_metrics = {
    "CoLA":"MCC",
    "MNLI":"Acc.",
    "MNLI-MM":"Acc.",
    "MRPC":"F$_1$",
    "QNLI":"Acc.",
    "QQP":"F$_1$",
    "RTE":"Acc.",
    "SST-2":"Acc.",
    "STS-B":"$\\rho$"
}

In [15]:
print("\\begin{center}\n\t\\scalebox{0.7}{\n\t\t\\begin{tabular}{rc|ccccccccccccc} \n\t\t\\toprule[1pt] \n\t\t\\multirow{2}{*}{Task (metric)} & \\multirow{2}{*}{Baseline} & \\multicolumn{13}{c}{Fine-tuned layers} \\\\ \n\t\t\\cline{3-15} \n\t\t& & 12-23 & 13-23 & 14-23 & 15-23 & 16-23 & 17-23 & 18-23 & 19-23 & 20-23 & 21-23 & 22-23 & 23-23 & None \\\\ \n\t\t\t\\midrule")

avg_performance = []

for task in tasks:
    m = metrics[task][0]
    base_key = f"base-{m}"
    
    if task == "MNLI-MM":
        row = f"\t\t\tMNLI-mm ({latex_metrics[task]}) & "
    else:
        row = f"\t\t\t{task} ({latex_metrics[task]}) & "
    
    row += "{:0.2f}".format(round(results[task][base_key] * 100, 2))
    
    for ind, val in enumerate(results[task][m]):
        row += " & {:0.2f}".format(round(val * 100,2))
        
        if len(avg_performance) == ind:
            avg_performance.append([])
            
            
        percent = (val / results[task][base_key]) * 100
        avg_performance[ind].append(percent)
        
#     row += "& {}".format(results[task]["90%"])
#     row += "& {}".format(results[task]["95%"])
        
    row += " \\\\"
    print(row)
    
print("\t\t\t\\midrule\\midrule")

row = "\t\t\tRel. perf. (\%) & 100.00"

for perf in avg_performance:
    row += " & {:0.2f}".format(round(np.mean(perf) ,2))
    
row += " \\\\"

print(row)
    
print("\t\t\\end{tabular}\n\t}\n\t\\caption{MTDNN-RoBERTa-large on GLUE}\n\t\\label{table:finetune-all}\n\\end{center}")

\begin{center}
	\scalebox{0.7}{
		\begin{tabular}{rc|ccccccccccccc} 
		\toprule[1pt] 
		\multirow{2}{*}{Task (metric)} & \multirow{2}{*}{Baseline} & \multicolumn{13}{c}{Fine-tuned layers} \\ 
		\cline{3-15} 
		& & 12-23 & 13-23 & 14-23 & 15-23 & 16-23 & 17-23 & 18-23 & 19-23 & 20-23 & 21-23 & 22-23 & 23-23 & None \\ 
			\midrule
			CoLA (MCC) & 65.91 & 60.10 & 59.84 & 60.27 & 62.32 & 61.87 & 61.49 & 60.80 & 60.53 & 58.86 & 58.69 & 54.72 & 48.95 & 0.00 \\
			SST-2 (Acc.) & 95.55 & 95.74 & 95.49 & 95.16 & 95.20 & 95.05 & 95.11 & 94.69 & 94.75 & 93.85 & 93.58 & 94.11 & 92.32 & 79.27 \\
			MRPC (F$_1$) & 92.79 & 92.64 & 92.43 & 92.05 & 91.39 & 91.46 & 91.66 & 91.31 & 90.02 & 86.25 & 84.80 & 85.22 & 84.20 & 81.22 \\
			STS-B ($\rho$) & 91.91 & 90.95 & 90.77 & 90.46 & 90.53 & 90.26 & 89.83 & 88.67 & 87.88 & 85.76 & 84.29 & 83.54 & 82.38 & 11.19 \\
			RTE (Acc.) & 85.05 & 83.03 & 81.65 & 80.96 & 80.58 & 78.64 & 76.90 & 73.65 & 69.25 & 66.61 & 64.20 & 61.49 & 59.45 & 48.74 \\
			\midrule\midru